# Deploying published model

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import pyspark.sql.functions as psqlf
from pyspark.ml.feature import VectorAssembler, Normalizer, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps
import urllib3, requests, json, base64, ibm_boto3
from ibm_botocore.client import Config 

### Information about published models from Watson ML instance

In [39]:
# @hidden_cell
#hide credentials
wml_credentials = {
  "apikey": "RTClG03LkiAX5Smoi2JChk31vEYd6hKfwiz3bbsilBGK",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/a25876fc4dfd4284aa7fa16810798644:ad345490-866c-4fcd-bbbc-a4d81c00b8bf::",
  "iam_apikey_name": "auto-generated-apikey-989e787a-6cd2-4d3f-93b9-891ca7239b8e",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/a25876fc4dfd4284aa7fa16810798644::serviceid:ServiceId-a3c740ff-ceb5-41e4-8e27-48540cfd6d8a",
  "instance_id": "ad345490-866c-4fcd-bbbc-a4d81c00b8bf",
  "password": "5efdcb6d-a110-40ae-b068-58bc7e2ca421",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "989e787a-6cd2-4d3f-93b9-891ca7239b8e"
}

In [26]:
headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{url}/v3/identity/token'.format(url=wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

Get info about Watson ML instance

In [27]:
endpoint_instance = wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id']
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)

print(response_get_instance)
#print(json.dumps(response_get_instance.text, indent=2))

<Response [200]>


Get URL of published models

In [28]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')
print(endpoint_published_models)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/ad345490-866c-4fcd-bbbc-a4d81c00b8bf/published_models


In [29]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
response_get = requests.get(endpoint_published_models, headers=header)
print(response_get)
#print(json.dumps(response_get.text, indent=2))

<Response [200]>


List of published models

In [30]:
list_of_published_models = [x.get('metadata').get('url') for x in json.loads(response_get.text).get('resources')]
list_of_published_models

['https://us-south.ml.cloud.ibm.com/v3/wml_instances/ad345490-866c-4fcd-bbbc-a4d81c00b8bf/published_models/ffa62ad1-8721-4d93-a49c-7f5df33d6e5c',
 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/ad345490-866c-4fcd-bbbc-a4d81c00b8bf/published_models/34b10bb0-108b-4d20-961b-188fb56f51b1']

Deployment URL for latest model

In [31]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('url') == list_of_published_models[len(list_of_published_models)-1]]
print(endpoint_deployments)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/ad345490-866c-4fcd-bbbc-a4d81c00b8bf/published_models/34b10bb0-108b-4d20-961b-188fb56f51b1/deployments


No of deployments per model

In [32]:
[x.get('entity').get('deployments').get('count') for x in json.loads(response_get.text).get('resources')]

[1, 2]

### Configure deployment

In [10]:
# @hidden_cell
spark_credentials = {"tenant_id": "s895-0265e0bc4ba2c2-7809e8a30329",
  "tenant_id_full": "9d7fe4b3-df43-47eb-8895-0265e0bc4ba2_9e5cbf9c-ee75-4d56-80c2-7809e8a30329",
  "cluster_master_url": "https://spark.bluemix.net",
  "tenant_secret": "b87c64ae-3cbe-43e4-8c2e-f404fc5ff37d",
  "instance_id": "9d7fe4b3-df43-47eb-8895-0265e0bc4ba2",
  "plan": "ibm.SparkService.PayGoPersonal"}

In [11]:
# @hidden_cell
cos_credentials = {
  "apikey": "by2MqrK_KQvvFwNryGGaF3vMKnyHEoqf4zXj4I2RfBHs",
  "cos_hmac_keys": {
    "access_key_id": "ce23d1002c6645cb8fc17b5dbb477e33",
    "secret_access_key": "3a200e8fb87ba25cbc3a534d75a5eb6256c2ac15c12411d7"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/a25876fc4dfd4284aa7fa16810798644:0a0616e2-2cbf-43fc-8905-08e39898d3e7::",
  "iam_apikey_name": "auto-generated-apikey-ce23d100-2c66-45cb-8fc1-7b5dbb477e33",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/a25876fc4dfd4284aa7fa16810798644::serviceid:ServiceId-2725cd4a-70f1-44fb-b9d5-1bffe97559aa",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/a25876fc4dfd4284aa7fa16810798644:0a0616e2-2cbf-43fc-8905-08e39898d3e7::"
}

In [12]:
auth_endpoint = 'https://iam.bluemix.net/oidc/token'
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'
cos = ibm_boto3.client('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [13]:
connection_apikey = cos_credentials['apikey']
connection_resource_instance_id = cos_credentials["resource_instance_id"]
connection_access_key_id = cos_credentials['cos_hmac_keys']['access_key_id']
connection_secret_access_key = cos_credentials['cos_hmac_keys']['secret_access_key']

In [14]:
encoded_spark_credentials_header = base64.b64encode(json.dumps({'credentials': spark_credentials, 'version': "2.1"}).encode())
result_filename = "scores.csv"

In [15]:
header_batch = {'Content-Type': 'application/json', 'Authorization': "Bearer " + mltoken, 'X-Spark-Service-Instance': encoded_spark_credentials_header}

In [16]:
payload_batch = {"type":"batch", "name": "Fraudalent Transactions Prediction", "description": "Batch Deployment", "input": {"source": { "bucket": "modelscorebucket", \
"filename": "data_to_score.csv", "inferschema":1,  "fileformat": "csv", "type": "cloudobjectstorage"}, \
"connection": {"access_key": connection_access_key_id, "secret_key": connection_secret_access_key, "url": service_endpoint}}, \
                 "output": { "target": { "bucket": "modelscorebucket", "filename": result_filename, "fileformat": "csv", "firstlineheader":"true", \
            "type": "cloudobjectstorage"}, "connection": {"access_key": connection_access_key_id, "secret_key": connection_secret_access_key, "url": service_endpoint}}}


### Create batch deployment for published model

In [17]:
response_batch = requests.post(endpoint_deployments, json=payload_batch, headers=header_batch)

print(response_batch)
print(json.loads(response_batch.text).get("entity").get("status"))

<Response [201]>
INITIALIZING


In [20]:
header_monitor_job = {'Content-Type': 'application/json', 'Authorization': "Bearer " + mltoken, 'X-Spark-Service-Instance': encoded_spark_credentials_header}

response_monitor_job = requests.get(endpoint_deployments, headers=header_monitor_job)

print(response_monitor_job)
json.loads(response_monitor_job.text).get("resources")[0]['entity']['status_details']
#print(json.dumps(response_monitor_job.text, indent=2))

<Response [200]>


{'completionCode': '200',
 'logFile': 'SubmissionID = driver-20180901044652-0488-a1c508b5-e683-449b-a014-0f457af921e5',
 'message': 'SUCCESS',
 'queueDateTime': '2018-09-01T09:46:52Z',
 'startDateTime': '2018-09-01T09:46:52Z',
 'status': 'COMPLETED'}

### Check scored data

In [23]:
cos.download_file(Key="scores.csv",Bucket="modelscorebucket", Filename="scores_local.csv")

In [24]:
scores_df = pd.read_csv('scores_local.csv')
scores_df.head(5)

,folds,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,...,orgbalDiff,type_indexed,destNameChar_indexed,type_ohe,destNameChar_ohe,features,features_norm,rawPrediction,probability,prediction
0,3,1,PAYMENT,5086.48,C598357562,0.00,0.00,M1593224710,0.00,0.00,...,-5086.48,1.0,1.0,"(4,[1],[1.0])","(1,[],[])","(15,[1,4,10,11,12,13,14],[1.0,5086.48,1.0,10.0...","(15,[1,4,10,11,12,13,14],[1.1350667648400314E-...","[99.99956526608419,4.3473391580797283E-4]","[0.9999956526608419,4.3473391580797285E-6]",0.0
1,3,1,PAYMENT,1718.29,C603658030,117811.00,116092.71,M1689924104,0.00,0.00,...,0.00,1.0,1.0,"(4,[1],[1.0])","(1,[],[])","(15,[1,4,5,6,10,11,12,13],[1.0,1718.29,117811....","(15,[1,4,5,6,10,11,12,13],[6.0453200903120725E...","[99.97988904170549,0.02011095829451225]","[0.9997988904170549,2.011095829451225E-4]",0.0
2,3,1,PAYMENT,2803.83,C1129433283,0.00,0.00,M864138492,0.00,0.00,...,-2803.83,1.0,1.0,"(4,[1],[1.0])","(1,[],[])","(15,[1,4,10,11,12,13,14],[1.0,2803.83,1.0,11.0...","(15,[1,4,10,11,12,13,14],[2.0591388909739188E-...","[99.99956526608419,4.3473391580797283E-4]","[0.9999956526608419,4.3473391580797285E-6]",0.0
3,3,1,TRANSFER,2061082.82,C2007599722,0.00,0.00,C1590550415,14939915.42,19169204.93,...,-2061082.82,3.0,0.0,"(4,[3],[1.0])","(1,[0],[1.0])","[0.0,0.0,0.0,1.0,2061082.82,0.0,0.0,1.49399154...","[0.0,0.0,0.0,4.0694249420001595E-8,0.083874218...","[99.96101501074776,0.03898498925224239]","[0.9996101501074776,3.898498925224239E-4]",0.0
4,3,1,CASH_IN,35902.49,C839771540,371688.15,407590.65,C2001112025,49003.30,0.00,...,-71804.99,2.0,0.0,"(4,[2],[1.0])","(1,[0],[1.0])","[0.0,0.0,1.0,0.0,35902.49,371688.15,407590.65,...","[0.0,0.0,1.766835521248905E-6,0.0,0.0634337946...","[99.99593086002912,0.004069139970885306]","[0.9999593086002911,4.069139970885305E-5]",0.0


In [25]:
print("Confusion matrix")
pd.crosstab(index= scores_df['prediction'], columns=scores_df['isFraud'])

Confusion matrix


isFraud,0
prediction,
0.0,400
